In [1]:
import sys
import os
from  pathlib import  Path
sys.path.append(str(Path.cwd().parent))
from  config import OPENAI_API_KEY,NCBI_API_KEY,EMAIL
from  src.clinfoai.pubmed_engine import PubMedNeuralRetriever

# Using Clinfo.AI 

In this tutorial, we will go through each step of the Clinfo.AI workflow. Before we start, we need to set up a few things. 


### 1.- Setting up enviorment:
1.a.- Install the conda enviroment using the yml file provided.

``` conda env create -f environment.yaml ```

1.b.- Select your environment to run notebook. I recommend using VScode: 



### 2.- Creating Accounts

You will need at least one account and at most two (depending on how many calls/hour you plan to do):
* OPENAI account: If you start a free account for the first time, you will get $5 in API credits.
* NCBI_API_KEY: This is only necessary if you plan to make more than 10 calls per hour.


Once you have created both accounts  go to **src\config.py** file and: 

* Set OPENAI_API_KEY to your openAI API key

If you created an NCBI API account add your key and email in the following values: 
* NCBI_API_KEY 
* EMAIL
* 
Otherwise, leave them as None

In [2]:
# Make Sure you followed at least step 1-2 before running this cell.
from  config import OPENAI_API_KEY, NCBI_API_KEY, EMAIL
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


### 3.- Defining your own promts:
We have designed prompts for each step of Clinfo.ai Workflow, leaveriging the power of in-contex-learning. If you want to us your own promps you can edit them **src\prompts** otherwise we will use the default prompts:

In [7]:
PROMPS_PATH = os.path.join("src","clinfoai","prompts","PubMed","Architecture_1","master.json")
current_directory = os.getcwd()
print(current_directory)

/Users/holeminhthach/Documents/GitHub/Clinfo.AI


### 4.- Define Clinfo.AI LLM Backbone
Clinfo uses a chain of LLMs to summarize information, thus we need to define an LLM backbone. 

We will start with OpenAI models, however, if you have access to GPUs it is possible to use Clinfo.AI with vLLM to use OpenSource LLMs as backbones (check tutorial 3).

In [8]:
MODEL:str  = "gpt-3.5-turbo"
#MODEL:str = "Qwen/Qwen2-beta-7B-Chat"

### 5.- Init Clinfo+Pubmed Engnie
We have all the necessary data to start our clinfo+pubmed instance:

In [9]:
## 5.- Init Neural Retriever from path. 
# Do not change the path if you want to use base  prompts, otherwise specify your own prompt architecture

nrpm = PubMedNeuralRetriever(
    architecture_path = PROMPS_PATH,
    model             = MODEL,
    verbose           = False,
    debug             = False,
    open_ai_key       = OPENAI_API_KEY,
    email             = EMAIL)


Task Name: pubmed_query_prompt
------------------------------------------------------------------------

Task Name: relevance_prompt
------------------------------------------------------------------------

Task Name: summarization_prompt
------------------------------------------------------------------------

Task Name: synthesize_prompt
------------------------------------------------------------------------


# Let's start!

In [10]:
### Step 0 : Ask a question ###
QUESTION    = "What is the prevalence of COVID-19 in the United States?"
QUESTION    = "What tests are needed to diagnose Chronic Neutropenia?"


## STEP 1 (Search PubMed): Convert the question into a query using an LLM
# This returns a list of queries (containing MESH terms)
# These queries are used to retrieve articles from NCBI
# Once retrieved we collect a list article ids.
pubmed_queries, article_ids = nrpm.search_pubmed(
    question=QUESTION,
    num_results=10,
    num_query_attempts=1)

print(f"Articles retrived: {len(article_ids)}")
print(pubmed_queries)
print(article_ids)

/Users/holeminhthach/Library/Caches/pypoetry/virtualenvs/clinfo-ai-CU3KyI9c-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/holeminhthach/Library/Caches/pypoetry/virtualenvs/clinfo-ai-CU3KyI9c-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Articles retrived: 10
['("Chronic Neutropenia" AND "diagnosis" AND "tests")']
['30870474', '3534197', '24827415', '10388004', '27841775', '6602565', '19305028', '20301576', '11964321', '34303547']


In [11]:
## Step 2: Fetch article data
# Preiously, we only extracted he PMIDs. No we will use those  PMIDs to retrive the metadata:
articles = nrpm.fetch_article_data(article_ids)

# Print example for first article: 
article_num = 1
print(f"Article {article_num}:\n")

#print(articles[article_num].keys())
#print(articles[article_num]['PubmedData'])
print(articles[article_num]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"])
#print(articles[article_num]["MedlineCitation"]["Article"])


Article 1:

['Neutrophil antibodies were demonstrated in 119 of 121 infants and young children with chronic neutropenia, establishing the diagnosis of autoimmune neutropenia of infancy. The median age at diagnosis was 8 months (range 3 to 30 months), and the female/male ratio was 6:4. Autoimmune neutropenia of infancy was manifested by recurrent fever and infection. All patients had selective neutropenia (absolute neutrophil count 0 to 500), and many had monocytosis. Fifteen of 16 patients tested failed to respond to epinephrine and hydrocortisone stimulation. Bone marrow had myeloid hyperplasia and reduced mature neutrophils. Recovery occurred in all 81 patients who passed the age of 5 years, except for one patient who is recovering at 6 1/2 years. The median age at recovery was 30 months; 95% recovered before 4 years. The estimated median duration of disease was 20 months. Neutrophil antibodies were detected early in the neutropenic phase by a combination of immunofluorescence and ag

In [12]:
# STEP 3 Summarize each article
# This step is parallelized, though it might look like one single call, it performs one call per article to summarize.
# Then the relevancy of the article (based on the original question) is provided by another LLM call.

article_summaries,irrelevant_articles =  nrpm.summarize_each_article(articles, QUESTION)

In [13]:
# Summaries for relevant articles
article_summaries

[{'title': '[Neutropenia in infancy - sometimes chronic and benign - own experiences].',
  'url': 'https://pubmed.ncbi.nlm.nih.gov/19305028/',
  'abstract': 'Chronic neutropenia is a decrease in circulating neutrophils in the peripheral blood lasting over 6 months. Values need to be refered with the age and race. In children aged 2 weeks to 12 months reffered values are above 1000/03BCL. There are congenital and aquired reasons of neutropenia in infancy. The most common type of chronic neutropenia in infants is chronic, benign neutropenia (AIN). Authors present ten infants between three and six months with chronic, benign neutropenia. The reason of ordering laboratory tests at outpatient clinic were benign upper respiratory tract infections (four cases), pallor (four cases) and on parental demand (one case). In one infant neutropenia was observed during treatment of pneumonia at a district hospital.',
  'citation': 'Kaczorowska-Hać B, Wierzba J, Stefanowicz J, Sielachowicz K, Wlazłowsk

In [14]:
# Articles deemed irelevant
irrelevant_articles 

[{'title': 'Invasive aspergillosis and endocarditis.',
  'url': 'https://pubmed.ncbi.nlm.nih.gov/34303547/',
  'abstract': 'INTRODUCTION:\nAspergillusfumigatus can cause a systemic infection called invasive aspergillosis causing pulmonary and extra-pulmonary damage. Aspergillus endocarditis (AE) is a relatively rare disease but can be life-threatening.\n\nCASE REPORTS:\nWe report here on five cases of endocarditis due to invasive aspergillosis: a 58-year-old man receiving immunosuppressive medication following a kidney graft, a 58-year-old man undergoing chemotherapy for chronic lymphocytic leukaemia, a 55-year-old man receiving corticosteroids for IgA vasculitis, a 52-year-old HIV-infected woman under no specific treatment and a 17-year-old boy under immunosuppressive therapy for auto-immune chronic neutropenia.\n\nDISCUSSION:\nAspergillus accounts for 25-30% of fungal endocarditis and 0.25% to 8.5% of all cases of infectious endocarditis. Aspergillus endocarditis results from invasio

In [15]:
# STEP 4 do a synthesis of all summaries to answer question: 
synthesis =   nrpm.synthesize_all_articles(article_summaries, QUESTION)
print("synthesis")
print(synthesis)

synthesis
Literature Summary: Chronic neutropenia encompasses various etiologies, including autoimmune neutropenia and chronic idiopathic neutropenia, with distinct diagnostic approaches and prognoses. Studies have highlighted the importance of tests such as bone marrow aspiration, biopsy, CFU-GM and CSA assays, and antineutrophil antibody quantification in diagnosing and predicting the clinical course of chronic neutropenia. While autoimmune neutropenia is associated with anti-neutrophil antibodies, myeloid lineage maturation arrest, and temporary remission with intravenous IgG therapy, chronic idiopathic neutropenia tends to remit more frequently in children than in adults. Granulocyte colony-stimulating factor (G-CSF) shows efficacy in increasing neutrophil counts, particularly in cases with recurrent infections.

TL;DR: Diagnostic tests for chronic neutropenia include bone marrow studies, CFU-GM and CSA assays, and antineutrophil antibody quantification, with G-CSF being effective 

# Great! We answered our first question using Clinfo.AI!
## Here are all the steps condensed:

In [18]:
PROMPS_PATH  = os.path.join("src","clinfoai","prompts","PubMed","Architecture_1","master.json")
MODEL:str    = "gpt-3.5-turbo"
# MODEL:str    = "Qwen/Qwen2-beta-7B-Chat"

nrpm = PubMedNeuralRetriever(
    architecture_path = PROMPS_PATH,
    model             = MODEL,
    verbose           = False,
    debug             = False,
    open_ai_key       = OPENAI_API_KEY,
    email             = EMAIL)

### STEP 0: Ask a question ###
question    = "What is the prevalence of COVID-19 in the United States?"

### STEP 1: Search PubMed ###
pubmed_queries, article_ids = nrpm.search_pubmed(question,num_results=10,num_query_attempts=1)

### STEP 2: Fetch article data ###
articles = nrpm.fetch_article_data(article_ids)

### STEP 3 Summarize each article (only if they are relevant [Step 3]) ###
article_summaries,irrelevant_articles =  nrpm.summarize_each_article(articles, question)


### STEP 4: Synthesize the results ###
synthesis =   nrpm.synthesize_all_articles(article_summaries, question)
print("Result:")
print(synthesis)


Task Name: pubmed_query_prompt
------------------------------------------------------------------------

Task Name: relevance_prompt
------------------------------------------------------------------------

Task Name: summarization_prompt
------------------------------------------------------------------------

Task Name: synthesize_prompt
------------------------------------------------------------------------
Result:
Literature Summary: Studies examining COVID-19 prevalence in the United States indicate varying rates of infection and outcomes. A mathematical modeling study on the effects of vaccination campaigns showed significant reductions in COVID-19 deaths across all 50 states and the District of Columbia [1]. A cross-sectional survey found a 1.7% positivity rate for COVID-19 among adults in the US, with common symptoms including dry cough, fever, and shortness of breath [3]. Retrospective analyses on neonatal COVID-19 cases reported a 0.1% incidence rate, with 7.7% of cases bei